In [3]:
%load_ext sparksql_magic
from pyspark.sql import SparkSession

# Start Spark session
spark = (
    SparkSession.builder.appName("FilterGHJsonPYTHON")
    .config("spark.sql.catalogImplementation", "hive")
    .enableHiveSupport()
    .getOrCreate()
)

In [2]:
df = spark.read.json("../datap/*.json.gz")
df = df.filter(df["type"].isin("IssuesEvent", "IssueCommentEvent"))
df.createOrReplaceTempView("df")

25/02/20 22:35:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [17]:
%%sparksql
WITH opened_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        MIN(to_timestamp(created_at)) AS opened_at,  -- First opened or reopened event
        FIRST(payload.issue.author_association) AS author_association,  
        FIRST(payload.issue.title) AS title,
        FIRST(payload.issue.labels) AS labels,
        FIRST(payload.issue.state) AS state,
        FIRST(payload.issue.state_reason) AS state_reason,
        FIRST(payload.issue.body) AS body
    FROM df
    WHERE type = 'IssuesEvent' 
      AND (payload.action = 'opened' OR payload.action = 'reopened')  -- Only take these events
    GROUP BY payload.issue.id
), 

closed_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        MAX(to_timestamp(created_at)) AS closed_at  -- Last closed event
    FROM df
    WHERE type = 'IssuesEvent' 
      AND payload.action = 'closed'  -- Only take closed events
    GROUP BY payload.issue.id
)

SELECT 
    o.issue_id,
    o.opened_at,
    c.closed_at,
    (unix_timestamp(c.closed_at) - unix_timestamp(o.opened_at)) / 3600 AS time_to_close_hours,
    o.author_association,
    o.title,
    o.state,
    o.state_reason,
    o.body
FROM opened_issues o
JOIN closed_issues c 
ON o.issue_id = c.issue_id
WHERE c.closed_at >= o.opened_at AND o.state != 'open' -- Ensure closed_at is after opened_at
ORDER BY time_to_close_hours DESC;


only showing top 20 row(s)


issue_id,opened_at,closed_at,time_to_close_hours,author_association,title,state,state_reason,body
2765157628,2025-01-01 11:38:02,2025-01-01 11:38:33,0.008611111111111111,NONE,issue-reopener-subdir pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1442 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572509435/attempts/1)
2765157618,2025-01-01 11:38:02,2025-01-01 11:38:32,0.008333333333333333,NONE,issue-reopener-path pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1442 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572509435/attempts/1)
2765157679,2025-01-01 11:38:09,2025-01-01 11:38:38,0.008055555555555555,NONE,issue-reopener-path pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1443 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572510530/attempts/1)
2765157609,2025-01-01 11:38:00,2025-01-01 11:38:27,0.0075,NONE,issue-reopener-basic pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1442 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572509435/attempts/1)
2765157684,2025-01-01 11:38:10,2025-01-01 11:38:35,0.006944444444444444,NONE,issue-reopener-vanityURLs pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1443 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572510530/attempts/1)
2765157704,2025-01-01 11:38:13,2025-01-01 11:38:35,0.006111111111111111,NONE,issue-reopener-subdir pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1443 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572510530/attempts/1)
2765157757,2025-01-01 11:38:20,2025-01-01 11:38:36,0.0044444444444444444,NONE,issue-reopener-basic pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1444 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572511422/attempts/1)
2765161782,2025-01-01 11:48:15,2025-01-01 11:48:31,0.0044444444444444444,NONE,issue-reopener-vanityURLs pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1466 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572579639/attempts/1)
2765157823,2025-01-01 11:38:29,2025-01-01 11:38:44,0.004166666666666667,NONE,issue-reopener-vanityURLs pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1444 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572511422/attempts/1)
2765161505,2025-01-01 11:47:39,2025-01-01 11:47:54,0.004166666666666667,NONE,issue-reopener-subdir pre-submit test issue,closed,completed,Test issue for commit https://github.com/ianlewis/todo-issue-reopener/pull/1460 - [Workflow run](https://github.com/ianlewis/todo-issue-reopener/actions/runs/12572575247/attempts/1)


In [18]:
%%sparksql
WITH opened_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        MIN(to_timestamp(created_at)) AS opened_at,  -- First opened or reopened event
        FIRST(payload.issue.author_association) AS author_association,  
        FIRST(payload.issue.title) AS title,
        FIRST(payload.issue.labels) AS labels,
        FIRST(payload.issue.state) AS state,
        FIRST(payload.issue.state_reason) AS state_reason,
        FIRST(payload.issue.body) AS body
    FROM df
    WHERE type = 'IssuesEvent' 
      AND (payload.action = 'opened' OR payload.action = 'reopened')  -- Only take these events
    GROUP BY payload.issue.id
), 

closed_issues AS (
    SELECT 
        payload.issue.id AS issue_id, 
        MAX(to_timestamp(created_at)) AS closed_at  -- Last closed event
    FROM df
    WHERE type = 'IssuesEvent' 
      AND payload.action = 'closed'  -- Only take closed events
    GROUP BY payload.issue.id
)

SELECT 
    o.issue_id,
    o.opened_at,
    c.closed_at,
    (unix_timestamp(c.closed_at) - unix_timestamp(o.opened_at)) / 3600 AS time_to_close_hours,
    o.author_association,
    o.title,
    o.state,
    o.state_reason,
    o.body
FROM opened_issues o
JOIN closed_issues c 
ON o.issue_id = c.issue_id
WHERE c.closed_at >= o.opened_at  -- Ensure the close happened after the open
ORDER BY time_to_close_hours DESC;


only showing top 20 row(s)
